# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [4]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

C:\OnlineSchool\Udacity\DataEngineer\Project2


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [5]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [6]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [41]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [42]:
# TO-DO: Create a Keyspace 
query = """
        CREATE KEYSPACE IF NOT EXISTS sparkify_analytics
        WITH REPLICATION = {'class': 'SimpleStrategy', 'replication_factor': 1}
        """
try:
    session.execute(query)
    print("Keyspace 'sparkify_analytics' is created!")
except Exception as e:
    print("Create keyspace encountered exception:")
    print(e)

Keyspace 'sparkify_analytics' is created!


#### Set Keyspace

In [43]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace("sparkify_analytics")
    print("Keyspace is set to 'sparkify_analytics'.")
except Exception as e:
    print("Exception encountered while setting keyspace:")
    print(e)

Keyspace is set to 'sparkify_analytics'.


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [44]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
query1 = """
         CREATE TABLE IF NOT EXISTS song_plays_by_session
         (sessionId INT, itemInSession INT,artist TEXT, song TEXT, length DECIMAL, PRIMARY KEY(sessionId, itemInSession))
         """

try:
    session.execute(query1)
    print("'song_plays_by_session' table is created.")
except Exception as e:
    print("Exception encountered while creating 'song_plays_by_session' table:")
    print(e)
                    

'song_plays_by_session' table is created.


In [45]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    #csvreader = csv.reader(f)
    csvreader = csv.DictReader(f)
    
    #next(csvreader) # skip header
    
    for line in csvreader:
        ## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO song_plays_by_session (sessionId, itemInSession, artist, song, length)"
        query = query + "VALUES(%s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        #session.execute(query, (line[8], line[#]))
        session.execute(query, (int(line['sessionId']), int(line['itemInSession']), line['artist'], line['song'], float(line['length'])))

print("Insert statement into 'song_plays_by_session' is completed!")

Insert statement into 'song_plays_by_session' is completed!


#### Do a SELECT to verify that the data have been inserted into each table

In [46]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
query = "SELECT * FROM song_plays_by_session WHERE sessionId = 139 AND itemInSession = 1"
try:
    rows = session.execute(query)
    for row in rows:
        print(row)
except Exception as e:
    print(e)

Row(sessionid=139, iteminsession=1, artist="Des'ree", length=Decimal('246.30812'), song='You Gotta Be')


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [47]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
query2 = """
         CREATE TABLE IF NOT EXISTS song_plays_by_user
         (userId INT, sessionId INT, itemInSession INT,artist TEXT, song TEXT, firstName TEXT, lastName TEXT, 
          PRIMARY KEY((userId, sessionId),itemInSession)
         )
         """

try:
    session.execute(query2)
    print("'song_plays_by_user' table is created.")
except Exception as e:
    print("Exception encountered while creating 'song_plays_by_user' table:")
    print(e)

                    

'song_plays_by_user' table is created.


In [48]:
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    #csvreader = csv.reader(f)
    csvreader = csv.DictReader(f)
    
    #next(csvreader) # skip header
    
    for line in csvreader:
        ## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO song_plays_by_user (userId, sessionId, itemInSession, artist, song, firstName, lastName)"
        query = query + "VALUES(%s, %s, %s, %s, %s,%s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        #session.execute(query, (line[8], line[#]))
        session.execute(query, (int(line['userId']),int(line['sessionId']), int(line['itemInSession']), line['artist'], line['song'], line['firstName'],line['lastName']))

print("Insert statement into 'song_plays_by_user' is completed!")

Insert statement into 'song_plays_by_user' is completed!


In [49]:
query = "SELECT itemInSession, artist, song, firstname , lastname  FROM song_plays_by_user WHERE userId = 8 AND sessionId = 139";
try:
    rows = session.execute(query)
    for row in rows:
        print(row)
except Exception as e:
    print(e)

Row(iteminsession=1, artist="Des'ree", song='You Gotta Be', firstname='Kaylee', lastname='Summers')
Row(iteminsession=3, artist='Mr Oizo', song='Flat 55', firstname='Kaylee', lastname='Summers')
Row(iteminsession=4, artist='Tamba Trio', song='Quem Quiser Encontrar O Amor', firstname='Kaylee', lastname='Summers')
Row(iteminsession=5, artist='The Mars Volta', song='Eriatarka', firstname='Kaylee', lastname='Summers')
Row(iteminsession=6, artist='Infected Mushroom', song='Becoming Insane', firstname='Kaylee', lastname='Summers')
Row(iteminsession=7, artist='Blue October / Imogen Heap', song='Congratulations', firstname='Kaylee', lastname='Summers')
Row(iteminsession=8, artist='Girl Talk', song='Once again', firstname='Kaylee', lastname='Summers')


In [50]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query3 = """
         CREATE TABLE IF NOT EXISTS users_by_song
         (userId INT, song TEXT, firstName TEXT, lastName TEXT, 
          PRIMARY KEY(song, userId)
         )
         """

try:
    session.execute(query3)
    print("'users_by_song' table is created.")
except Exception as e:
    print("Exception encountered while creating 'users_by_song' table:")
    print(e)

                    

'users_by_song' table is created.


In [52]:
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    #csvreader = csv.reader(f)
    csvreader = csv.DictReader(f)
    
    #next(csvreader) # skip header
    
    for line in csvreader:
        ## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO users_by_song (userId, song, firstName, lastName)"
        query = query + "VALUES(%s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        #session.execute(query, (line[8], line[#]))
        session.execute(query, (int(line['userId']), line['song'], line['firstName'],line['lastName']))

print("Insert statement into 'users_by_song' is completed!")

Insert statement into 'users_by_song' is completed!


In [53]:
query = "SELECT firstname , lastname  FROM users_by_song WHERE song='All Hands Against His Own'";
try:
    rows = session.execute(query)
    for row in rows:
        print(row)
except Exception as e:
    print(e)

Row(firstname='Jacqueline', lastname='Lynch')
Row(firstname='Tegan', lastname='Levine')
Row(firstname='Sara', lastname='Johnson')


### Drop the tables before closing out the sessions

In [54]:
## TO-DO: Drop the table before closing out the sessions
drop_table_queries = ["DROP TABLE IF EXISTS song_plays_by_session", 
                      "DROP TABLE IF EXISTS song_plays_by_user",
                      "DROP TABLE IF EXISTS users_by_song"];

for query in drop_table_queries:
    session.execute(query)
    print("Executed: " + query)
    
print("Drop table completed!")

Executed: DROP TABLE IF EXISTS song_plays_by_session
Executed: DROP TABLE IF EXISTS song_plays_by_user
Executed: DROP TABLE IF EXISTS users_by_song
Drop table completed!


### Close the session and cluster connection¶

In [55]:
session.shutdown()
cluster.shutdown()